### Задача

Исследование многокомнатного помещения сенсором ограниченного обзора за минимальное число шагов


### Базовая имплементация

- Env: Dungeon

- Награда за шаг: процент исследованных клеток на данном шаге / общее кол-во клеток

Необходимо оценить качество текущей реализации и установить потенциальные слабые места. Будем отслеживать величины, как отвечающие за тренировку, так и описывающие единичную траекторию (в момент, когда мы генерируем траекторию в режиме no_grad для gif-изображения).

Тренировка:
- Средняя длина эпизода
- Средняя награда за эпизод
- Номер итерации, с которой обучение выходит на плато

Единичая траектория:
- Процент исследованных клеток
- Процент неправильных действий (moved)
- Среднее кол-во исследованных клеток за шаг

### Эксперимент № 1

Из графиков видно следующее:
    
- обучение выходит на плато на X итерации
- средняя награда за эпизод на последних этапах обучения ...

--добавить графики

### Идеи по улучшению награды


#### Базовые улучшения

- добавлять штраф за moved == False

- дисконтировать награду

- добавлять награду за досрочно завершенный эпизод (сама по себе данная награда никак не поможет, так как агент пока что никогда не завершает эпизод досрочно. Однако, если все-таки агент научится досрочно завершать эпизод с помощью других улучшений, данный пункт потенциально может подкрепить такое поведение агента (?) )


#### Другие идеи

- добавлять награду за процент занятых клеток среди увиденных на данном шаге. Судя по gif, часто агент не доходит до конца комнаты, и уходит в другую сторону, из-за чего в будущем ему приходется возвращаться на то же место. Возможно, такая награда в совокупности с дисконтированием заставит агента доходить до конца комнаты.

- ....

### Эксперимент № 2

Из графиков видно следующее

- добавить графики